In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

import utils as U

diode_path = Path("/home/gustavo/workstation/depth_estimation/data/datasets_quali/DIODE/")
diode_indoor = diode_path / "val" / "indoors"
diode_outdoor = diode_path / "val" / "outdoor"
diode_preds = Path("/home/gustavo/workstation/depth_estimation/data/outputs/DIODE-Anythingv2/npy/")

In [2]:
def eval_diode(gt_path, mask_path, pred_path):

    filenames_depth = U.get_sorted_files(gt_path, "_depth.npy")
    filenames_mask = U.get_sorted_files(mask_path, "_depth_mask.npy")
    filenames_preds = U.get_sorted_files(pred_path, ".npy")

    abs_rel_total = 0
    delta_total = 0

    num_imgs = len(filenames_depth)

    data = []

    for index in range(num_imgs):

        pred_diode = np.load(filenames_preds[index])
        groundtruth = np.squeeze(np.load(filenames_depth[index]))
        H,W = groundtruth.shape

        mask = np.load(filenames_mask[index]).reshape((H, W))
        mask_pred = np.ones_like(pred_diode)
        mask_pred[pred_diode < 1e-1] = 0
        mask_full = mask * mask_pred

        gt_depth_masked = np.zeros_like(groundtruth)
        gt_depth_masked[mask_full == 1] = groundtruth[mask_full == 1]

        gt_disp_masked = np.zeros_like(groundtruth)
        gt_disp_masked[mask_full == 1] = 1.0 / groundtruth[mask_full == 1]

        x_0, x_1 = U.compute_scale_and_shift(pred_diode, gt_disp_masked, mask_full)

        prediction_aligned = x_0 * pred_diode + x_1
        prediction_aligned = np.squeeze(prediction_aligned)*mask_full

        depth_aligned_masked = np.zeros_like(pred_diode)
        depth_aligned_masked[mask_full == 1] = 1 / prediction_aligned[mask_full == 1]

        abs_rel = U.abs_rel_error_mask(gt_depth_masked,depth_aligned_masked , mask_full)
        delta = U.calculate_delta(gt_depth_masked, depth_aligned_masked , mask_full, threshold=1.25)

        abs_rel_total = abs_rel_total + abs_rel
        delta_total = delta_total + delta

        row = {"fname": index, "absrel": abs_rel, "delta": delta}
        data.append(row)
    
    return abs_rel_total / num_imgs, delta_total/num_imgs, pd.DataFrame(data)

In [3]:
absrel, delta, df = eval_diode(diode_path, diode_path, diode_preds)

In [7]:
absrel

np.float32(0.18034954)

In [8]:
delta

np.float64(75.72203511895378)

In [4]:
df.head()

,fname,absrel,delta
0,0,0.052337,95.337142
1,1,0.036899,98.428650
2,2,0.074680,94.016174
3,3,0.027188,98.408820
4,4,0.028566,99.080527


In [5]:
df.tail()

,fname,absrel,delta
766,766,0.388852,22.059645
767,767,0.136647,10.384759
768,768,0.219132,23.825043
769,769,0.126241,32.829876
770,770,0.170131,25.853179


In [6]:
df.to_csv("diode_results.csv", index=False)